# Practice session on pandas

Reminder: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [1]:
import numpy as np
import pandas as pd

# Bike rentals in Aschaffenburg

Each row of the dataframe `trips` describes a journey of a rental bike in the Aschaffenburg area.

(Of course, since the data is generated at random, the rental times do not necessarily match the true travel times.)

In [83]:
rng = np.random.default_rng(seed=42)
n = 100

trips = pd.DataFrame({
    "customer_id": 1000000 + rng.integers(low=0, high=50, size=n),
    "date": pd.Timestamp('2024-01-01') + pd.to_timedelta(rng.integers(low=0, high=366, size=n), unit='D'),
    "start location": rng.choice(["Town Hall", "Schloss Johannisburg", "Park Schönbusch", "Pompejanum", "Public Library", "University"], size=n),
    "end location": rng.choice(["Town Hall", "Schloss Johannisburg", "Park Schönbusch", "Pompejanum", "Public Library", "University"], size=n),
    "minutes": rng.exponential(scale=50, size=n).round().astype(int)
})

print("number of rows: ", len(trips))
trips.head()

number of rows:  100


,customer_id,date,start location,end location,minutes
0,1000004,2024-10-31,Park Schönbusch,Park Schönbusch,22
1,1000038,2024-03-14,University,Public Library,103
2,1000032,2024-10-21,Park Schönbusch,Town Hall,54
3,1000021,2024-01-03,Public Library,University,10
4,1000021,2024-10-18,Park Schönbusch,Pompejanum,0


a) What is the duration of the longest trip?

In [25]:
trips["minutes"].max()
np.max(trips["minutes"])
trips.sort_values("minutes").tail(1)

,customer_id,date,start location,end location,minutes
37,1000041,2024-01-12,Park Schönbusch,Schloss Johannisburg,215


b) How many trips ended at the castle "Schloss Johannisburg"?

In [30]:
np.sum(trips["end location"] == "Schloss Johannisburg")
len(trips[trips["end location"] == "Schloss Johannisburg"])
#trips[trips[trips["end location"] == "Schloss Johannisburg"]].shape(1)

22

c) What is the average duration of round trips, i.e. of trips with identical start and end location?

In [26]:
np.average(trips[trips["start location"] == trips["end location"]].minutes)

np.float64(60.8)

d) What is the number of unique end locations?

In [ ]:
trips["end location"].nunique()
len(trips.groupby("end location"))

6

e) Find the longest rental from the earliest day in `trips`

In [54]:
trips.sort_values(["date", "minutes"], ascending=[True, False]).head(1)
np.max(trips[trips["date"] == trips["date"].min()].minutes)

np.int64(10)

e) Show all the trips of the top customers, i.e. of those people having the highest number of rows. For this:

1. Find out the number of trips for each customer and build a dataframe containing this information.
2. Enrich `trips` by joining it with the dataframe from the first step.
3. Select the rows with the highest values. 

In [84]:
num_rentals = trips.groupby("customer_id").size().reset_index(name = "num rentals")
num_rentals
trips = pd.merge(left=num_rentals, right=trips, on="customer_id")
trips

,customer_id,num rentals,date,start location,end location,minutes
0,1000002,1,2024-08-30,Town Hall,University,26
1,1000003,3,2024-04-21,Public Library,Public Library,148
2,1000003,3,2024-12-29,Pompejanum,University,124
3,1000003,3,2024-05-02,Pompejanum,Town Hall,25
4,1000004,5,2024-10-31,Park Schönbusch,Park Schönbusch,22
...,...,...,...,...,...,...
95,1000046,2,2024-12-30,Public Library,University,4
96,1000047,1,2024-12-09,Schloss Johannisburg,Town Hall,5
97,1000048,3,2024-10-12,Public Library,Public Library,144
98,1000048,3,2024-11-08,Town Hall,Schloss Johannisburg,41


In [85]:
trips[trips["num rentals"] == trips["num rentals"].max()].sort_values(["customer_id", "date"])

,customer_id,num rentals,date,start location,end location,minutes
7,1000004,5,2024-02-03,Town Hall,Schloss Johannisburg,101
8,1000004,5,2024-03-01,Town Hall,Town Hall,18
6,1000004,5,2024-09-15,Public Library,Town Hall,34
5,1000004,5,2024-10-12,Public Library,Schloss Johannisburg,8
4,1000004,5,2024-10-31,Park Schönbusch,Park Schönbusch,22
84,1000041,5,2024-01-12,Park Schönbusch,Schloss Johannisburg,215
82,1000041,5,2024-03-30,Park Schönbusch,University,13
86,1000041,5,2024-04-20,Park Schönbusch,Town Hall,49
85,1000041,5,2024-09-11,Park Schönbusch,Schloss Johannisburg,16
83,1000041,5,2024-10-06,Pompejanum,Town Hall,75


# LEGO again

Start by reading in the file `LEGO.csv`. Then exclude merchandising products, i.e. the number of contained pieces should be at least one. 

In [105]:
lego = pd.read_csv("Lego/lego.csv")
lego = lego[~lego["pieceCount"].isna()].reset_index(drop = True)
print(lego.shape)
lego.head()

(7979, 12)


,name,theme,product_code,ageRange,pieceCount,price,minifigureCount,buildHeight,buildWidth,buildDepth,url,date
0,Eugens Museumssammlung,Animal Crossing™,77056,7+,543.0,79.99,NaN,NaN,NaN,NaN,/de-de/product/blatherss-museum-collection-77056,2025-10-07
1,Bienchens gemütliches Haus,Animal Crossing™,77058,6+,149.0,19.99,NaN,NaN,NaN,NaN,/de-de/product/goldies-cosy-house-77058,2025-10-07
2,Kreative Häuser: Jahreszeiten voller Spaß,Animal Crossing™,77057,7+,814.0,89.99,NaN,NaN,NaN,NaN,/de-de/product/creative-houses-seasons-of-fun-...,2025-10-07
3,Gerds Wohnwagen & Gärtnerei,Animal Crossing™,77054,7+,263.0,29.99,NaN,NaN,NaN,NaN,/de-de/product/leifs-caravan-garden-shop-77054,2025-10-07
4,K.K. spielt auf dem Festplatz,Animal Crossing™,77052,7+,550.0,79.99,NaN,NaN,NaN,NaN,/de-de/product/kks-concert-at-the-plaza-77052,2025-10-07


### Task 1: Number of sets in each webscrape

The dataset contains several webscrapes from different years. Find out whether the number of sets has generally stayed constant over time or whether there are more sets now than in the past.

### Task 2: Identify specific models

Your nephew's birthday is coming up. Since he loves playing police, you want to find all current models that contain the word "Polizei" in the name and cost at most 30€.

(Note that the name is of type `str`. A reference of str-specific functions can be found here:
https://pandas.pydata.org/docs/reference/series.html#string-handling Since the name should **contain** a specific word, you might easily guess the correct function.)

In [122]:
lego[lego["name"].str.contains('Polizei') & 
     (lego["price"] <= 30) & (lego["date"] == lego["date"].max())].reset_index(drop = True)

,name,theme,product_code,ageRange,pieceCount,price,minifigureCount,buildHeight,buildWidth,buildDepth,url,date
0,Verfolgungsjagd im Polizeiboot,City,60456,6+,264.0,29.99,NaN,NaN,NaN,NaN,/de-de/product/police-boat-chase-60456,2025-10-07
1,Verfolgungsjagd mit dem Polizeimotorrad,City,60455,4+,65.0,9.99,NaN,NaN,NaN,NaN,/de-de/product/police-motorcycle-chase-60455,2025-10-07
2,Polizeiauto,City,60312,5+,94.0,9.99,NaN,NaN,NaN,NaN,/de-de/product/police-car-60312,2025-10-07
3,Mobiles Polizeihunde-Training,City,60369,5+,197.0,19.99,NaN,NaN,NaN,NaN,/de-de/product/mobile-police-dog-training-60369,2025-10-07
4,Verfolgungsjagd mit Polizeiauto und Muscle Car,City,60415,6+,213.0,19.99,NaN,NaN,NaN,NaN,/de-de/product/police-car-and-muscle-car-chase...,2025-10-07
5,Polizeimotorrad,DUPLO®,10967,2+,5.0,9.99,NaN,NaN,NaN,NaN,/de-de/product/police-motorcycle-10967,2025-10-07


### Task 3

Which models are brand new to the product range, i.e. which sets occur only in the most recent webscrape?

In [123]:
#lego[(lego.groupby["name"].size() == 1) & (lego["date"] == lego["date"].max())]

occurances = lego.groupby("product_code").size().reset_index(name = "num_occurences")
lego = pd.merge(left = lego, right = occurances, on = "product_code")


In [125]:
lego[lego["num_occurences"] == 1 & (lego["date"] == lego["date"].max())]

,name,theme,product_code,ageRange,pieceCount,price,minifigureCount,buildHeight,buildWidth,buildDepth,url,date,num_occurences
0,Eugens Museumssammlung,Animal Crossing™,77056,7+,543.0,79.99,NaN,NaN,NaN,NaN,/de-de/product/blatherss-museum-collection-77056,2025-10-07,1
1,Bienchens gemütliches Haus,Animal Crossing™,77058,6+,149.0,19.99,NaN,NaN,NaN,NaN,/de-de/product/goldies-cosy-house-77058,2025-10-07,1
2,Kreative Häuser: Jahreszeiten voller Spaß,Animal Crossing™,77057,7+,814.0,89.99,NaN,NaN,NaN,NaN,/de-de/product/creative-houses-seasons-of-fun-...,2025-10-07,1
13,Schloss Neuschwanstein,Architecture,21063,18+,3455.0,269.99,NaN,NaN,NaN,NaN,/de-de/product/neuschwanstein-castle-21063,2025-10-07,1
25,Keith Haring – Tanzende Figuren,Art,31216,18+,1773.0,119.99,NaN,NaN,NaN,NaN,/de-de/product/keith-haring-dancing-figures-31216,2025-10-07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,Durrr Burger Restaurant,LEGO® Fortnite,77076,10+,546.0,59.99,NaN,NaN,NaN,NaN,/de-de/product/durrr-burger-restaurant-77076,2025-10-07,1
920,Zuckerguss-Gebirge und Kätzchen-Garten,LEGO® Gabbys Puppenhaus,11205,4+,263.0,49.99,NaN,NaN,NaN,NaN,/de-de/product/sweet-treat-mountain-kitty-gard...,2025-10-07,1
921,Meerjungfrau Gabbys Aquarium Abenteuer,LEGO® Gabbys Puppenhaus,11204,4+,103.0,19.99,NaN,NaN,NaN,NaN,/de-de/product/mermaid-gabbys-aquarium-adventu...,2025-10-07,1
925,Dorf Hogsmeade™ – Sammleredition,Harry Potter™,76457,18+,3228.0,379.99,12.0,33.0,76.0,14.0,/de-de/product/hogsmeade-village-collectors-ed...,2025-10-07,1


Which models have permanently stayed in the product range, i.e. which sets occur in each webscrape?

In [131]:
lego[lego["num_occurences"] == lego["num_occurences"].max()].sort_values(["product_code", "date"])

,name,theme,product_code,ageRange,pieceCount,price,minifigureCount,buildHeight,buildWidth,buildDepth,url,date,num_occurences
7375,Elementetrenner,Classic,630,4+,1.0,2.49,NaN,NaN,NaN,NaN,/de-de/product/brick-separator-630,2021-10-04,9
6693,Elementetrenner,Classic,630,4+,1.0,2.49,NaN,NaN,NaN,NaN,/de-de/product/brick-separator-630,2022-03-22,9
5844,Elementetrenner,Classic,630,4+,1.0,2.49,NaN,NaN,NaN,NaN,/de-de/product/brick-separator-630,2022-09-19,9
5046,Elementetrenner,Classic,630,4+,1.0,2.49,NaN,NaN,NaN,NaN,/de-de/product/brick-separator-630,2023-02-28,9
4003,Elementetrenner,Classic,630,4+,1.0,2.49,NaN,NaN,NaN,NaN,/de-de/product/brick-separator-630,2023-09-21,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,Connections Kit,SERIOUS PLAY®,2000431,6+,2455.0,559.99,NaN,NaN,NaN,NaN,/de-de/product/lego-serious-play-connections-k...,2023-09-21,9
3683,Connections Kit,SERIOUS PLAY®,2000431,6+,2455.0,559.99,NaN,NaN,NaN,NaN,/de-de/product/lego-serious-play-connections-k...,2024-03-28,9
2671,Connections Kit,SERIOUS PLAY®,2000431,6+,2455.0,559.99,NaN,NaN,NaN,NaN,/de-de/product/lego-serious-play-connections-k...,2024-10-12,9
1833,Connections Kit,SERIOUS PLAY®,2000431,6+,2455.0,559.99,NaN,NaN,NaN,NaN,/de-de/product/lego-serious-play-connections-k...,2025-04-17,9
